In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone



# Cargar el DataFrame
df = pd.read_csv("../data/clean_data/series.csv")  # Reemplaza con la ruta a tu archivo CSV
# Crea una instancia de Pinecone
pc = Pinecone(api_key='2c3b8e05-b4f8-4c38-82f1-6e41ac7fbcd1')  # Reemplaza con tu clave de API
index = pc.Index('series')
# Cargar modelo de embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')  # Modelo ligero para generar embeddings

embeddings = model.encode(df['embedding'].tolist())

# Insertar embeddings en Pinecone
for i, embedding in enumerate(embeddings):
    index.upsert([(df['IMDb ID'][i], embedding.tolist())]) # Almacena el vector en Pinecone con el ID del libro
print("Vectores cargados en Pinecone.")

/opt/anaconda3/envs/env_finalp/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


ProtocolError: Failed to connect; did you specify the correct index name?